# [ global ]

In [ ]:
# inbuilt 
import os
import sys
import math

# most common
import numpy as np
import matplotlib.pyplot as plt

# pytorch
import torch as tt
import torch.nn as nn
import torch.optim as oo
import torch.functional as ff
import torch.distributions as dd
import torch.utils.data as ud

# custom
import known
import known.ktorch as kt
from known.basic import pj
print(f'{sys.version=}\n{np.__version__=}\n{tt.__version__=}\n{known.__version__=}')

from torch.utils.data import Dataset, IterableDataset, DataLoader
import glob

import unicodedata
import string

import known 
import known.ktorch as kt

In [ ]:
vocab_words = ['a', 'b']
ds = kt.LangDataset(vocab_words, embed=1, dtype=tt.float32)

In [ ]:
known.basic.Verbose.show(ds)

In [ ]:
known.basic.Verbose.show(ds.vocab)

In [ ]:
ds.add_class('astr', 'bstr', 'mixed')


In [ ]:
ds.add_samples('astr', 'aaaa', 'aaa', 'aa')
ds.add_samples('bstr', 'bbbb', 'bbb', 'bb')
ds.add_samples('mixed', 'abab', 'aba', 'ba')

In [ ]:
dl=iter(DataLoader(ds, batch_size=1))
x,y = next(dl)


known.basic.Verbose.info(x, True)
known.basic.Verbose.info(y, True)


In [ ]:

rnnc = kt.ELMANX(
    input_size=ds.vocab.vlen,
    hidden_sizes=(32, 16, 8),
    output_sizes=(16, 8, ds.n_classes),
    dropout=0.0,
    batch_first=True,
    stack_output=True,
    cell_bias=True,
    out_bias=True,
    dtype=tt.float32,
    activation_gate=tt.sigmoid,
    activation_out=tt.tanh,
    activation_last=(nn.LogSoftmax,{'dim':-1})
)


In [ ]:

model=rnnc
epochs = 200
batch_size=1
shuffle=True
validation_freq = int(epochs/10)
criterion=nn.NLLLoss()
lr = 0.005
weight_decay = 0.0
optimizer=oo.Adam(rnnc.parameters(), lr=lr, weight_decay=weight_decay)
lrs=oo.lr_scheduler.LinearLR(optimizer, start_factor= 1.0, end_factor=0.7, total_iters=epochs)

early_stop_train=kt.QuantiyMonitor('TrainLoss', patience=50, delta=0.00001)
early_stop_val=kt.QuantiyMonitor('ValLoss', patience=50, delta=0.00001)
checkpoint_freq=int(epochs/4)
save_path='sample.rnn'
loss_plot_start = int(epochs/50)

trainer = kt.Trainer(model)
trainer.optimizer=optimizer
trainer.criterion=criterion

trainer.fit(training_data=ds, validation_data=None, 
            epochs=epochs, batch_size=batch_size,shuffle=shuffle,validation_freq=validation_freq,
            save_path=save_path, use_rnn=True, verbose=1)

trainer.plot_results(loss_plot_start=loss_plot_start)

#mtl, tl = trainer.evaluate(ds,use_rnn=True)
#print('loss', mtl)
print('=================================================')

In [ ]:
def predict(input_line):
    xaxis = ds.classes.keys()
    print('\n> %s' % input_line)
    with tt.no_grad():
        output, *_ = rnnc(tt.unsqueeze(tt.stack([ds.embed(s) for s in input_line]), dim=0))
        print(output.shape)
    
    output=tt.e**output.squeeze(0)
    #print(output)
    for i,ts in enumerate(output):
        plt.figure(figsize=(6,3))
        plt.ylim(0,1)
        plt.title(f'{i+1}')
        plt.bar(xaxis, ts)
        print(f'@{i} :: {ts}, {tt.sum(ts)}')
        plt.show()




In [ ]:
ds.data

In [ ]:
for xv,yv in ds.data:
    print(xv, 'true:', yv)
    predict(xv)
    